# 필요한 라이브러리 import

In [16]:
import csv
import pandas as pd
import os
import re
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict
import warnings
import pickle
plt.rc('font',family='Malgun Gothic')
plt.rc('font',size=10)
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.models import load_model
import tensorflow as tf
import joblib
warnings.filterwarnings(action='ignore')

In [17]:
with open('./models/defender/defender_qt.pickle','rb') as f:
    defender_po = pickle.load(f)

defender_model = load_model('./models/defender/defender_nn.h5')

premier_df = pd.read_csv('./datas/NEW_FM_DATA_MERGED/2021-2022/2021-2022-PremierLeague.csv')

In [18]:
premier_df.rename(columns={'Cr C/A':'크로스 성공률','FA':'파울 획득','Int/90':'인터셉트/90','공중 A/90':'공중볼시도/90','K Ps/90':'키패스/90','Cr A/90':'크로스 시도/90'},inplace=True)
premier_df = premier_df.astype({'골':np.int64,'도움':np.int64,'파울 획득':np.int64,'태클 성공률':np.int64})

In [19]:
defender_df = premier_df[premier_df['포지션']=='수비수']

In [20]:
# PSTL = Pass sucess try linear

defender_df['PSTL'] = (defender_df['경기당 패스 성공/90']) + (defender_df['경기당 패스 시도/90']*1.2)

In [21]:
independent_value = {'PSTL', '태클 성공/90', '헤더 성공/90'}

dependent_value = {'평균획득승점/경기'}

In [22]:
defender_df.loc[:,independent_value] = defender_po.transform(defender_df.loc[:,independent_value])

In [23]:
defender_df = defender_df.reset_index(drop=True)

In [24]:
defender_df['예측PPG']=defender_model.predict(defender_df.loc[:,independent_value])

In [25]:
defender_df.sort_values(by='예측PPG',ascending=False).head(5)

,이름,시즌,포지션,구단,리그,골,도움,임대 기간,승,무,...,경기당 패스 시도/90,경기당 패스 성공/90,키패스/90,태클 성공/90,드리블 성공/90,헤더 성공/90,크로스 시도/90,평균획득승점/경기,PSTL,예측PPG
17,Rodri,2021-2022,수비수,맨시티,잉글랜드 프리미어 리그,3,3,0,24,10,...,64.34,60.11,0.79,-2.693664,0.23,5.199338,0.11,1.82,2.826129,2.130419
18,Fernandinho,2021-2022,수비수,맨시티,잉글랜드 프리미어 리그,2,3,0,25,7,...,60.05,55.47,0.74,-2.683698,0.04,5.199338,0.20,1.95,1.351053,2.115969
22,Scott McTominay,2021-2022,수비수,맨 레드,잉글랜드 프리미어 리그,1,1,0,18,9,...,55.72,51.37,1.13,-2.472054,0.05,5.199338,0.93,1.91,-0.010334,2.022667
40,Thomas Partey,2021-2022,수비수,아스날,잉글랜드 프리미어 리그,4,3,0,22,2,...,55.15,47.46,1.48,-2.314181,0.13,5.199338,1.04,1.94,-0.561005,1.982092
59,Romain Saïss,2021-2022,수비수,울브스,잉글랜드 프리미어 리그,0,0,0,5,3,...,57.36,53.41,0.18,-3.436663,0.28,5.199338,0.28,1.20,0.408159,1.914391


In [26]:
defender_min = np.min(defender_df['예측PPG'].tolist())
defender_min

1.3122373819351196

In [27]:
defender_df['WAR'] = defender_df['예측PPG'].map(lambda x : x-defender_min+1)

In [28]:
defender_df.loc[:,independent_value] = defender_po.inverse_transform(defender_df.loc[:,independent_value])

In [29]:
defender_war = defender_df.sort_values(by='WAR',ascending=False).copy()
defender_war = defender_war.reset_index(drop=True)

In [30]:
defender_war[['이름','시즌','골', '도움','WAR']].sort_values(by='WAR',ascending=False).head(10)

,이름,시즌,골,도움,WAR
0,Rodri,2021-2022,3,3,1.818181
1,Fernandinho,2021-2022,2,3,1.803731
2,Scott McTominay,2021-2022,1,1,1.710430
3,Thomas Partey,2021-2022,4,3,1.669855
4,Romain Saïss,2021-2022,0,0,1.602154
5,Ben Davies,2021-2022,0,3,1.552043
6,Jonny,2021-2022,0,3,1.543294
7,Diego Carlos,2021-2022,1,1,1.541649
8,Jonny Evans,2021-2022,0,1,1.536778
9,Adam Masina,2021-2022,0,0,1.520837
